# Training the model

This notebook has as goal to train a model to classify `Chest X-Ray` images into `normal` or `pneumonia` classes. 



## Setup

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

# TensorFlow
import tensorflow as tf
from tensorflow import keras
from keras import utils, layers, optimizers, callbacks, metrics, Sequential

# Hugging Face
import datasets
from transformers import TFViTForImageClassification, ViTFeatureExtractor

In [2]:
DATASET_NAME = "mmenendezg/pneumonia_x_ray"
MODEL_NAME = "google/vit-large-patch32-384"

IMG_CLASSES = ["Normal", "Pneumonia"]
BATCH_SIZE = 32
SEED = 92
AUTOTUNE = tf.data.AUTOTUNE


## Load the dataset

In [3]:
def format_dataset(example: dict):
    return example["image"], example["label"]


def convert_tf_dataset(dataset: datasets.Dataset, shuffle: bool = False):
    tf_dataset = (
        dataset.to_tf_dataset(batch_size=1)
        .unbatch()
        .map(format_dataset, num_parallel_calls=AUTOTUNE)
    )
    if shuffle:
        tf_dataset.shuffle(dataset.num_rows)

    tf_dataset = tf_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

    return tf_dataset


def load_dataset():
    pneumonia_dataset = datasets.load_dataset(DATASET_NAME)
    train_ds = convert_tf_dataset(pneumonia_dataset["train"], shuffle=True)
    valid_ds = convert_tf_dataset(pneumonia_dataset["validation"])
    test_ds = convert_tf_dataset(pneumonia_dataset["test"])

    return train_ds, valid_ds, test_ds


In [4]:
train_ds, valid_ds, test_ds = load_dataset()


Found cached dataset parquet (/Users/mmenendezg/.cache/huggingface/datasets/mmenendezg___parquet/mmenendezg--pneumonia_x_ray-052017b06aabdb98/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

## Train the model

In [5]:
# Clear the TensorFlow session
tf.keras.backend.clear_session()
tf.random.set_seed(SEED)

In [6]:
feature_extractor = ViTFeatureExtractor.from_pretrained(MODEL_NAME)

data_augmentator = Sequential(
    [
        layers.Resizing(
            feature_extractor.size["height"],
            feature_extractor.size["width"],
            interpolation="nearest",
        ),
        layers.Rescaling(scale=1.0 / 255.0),
        layers.RandomFlip(mode="horizontal"),
        layers.RandomRotation(factor=0.2),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ]
)

# Load the pretrained model
id2label = {str(i): label for i, label in enumerate(IMG_CLASSES)}
label2id = {v: k for k, v in id2label.items()}
vit_model = TFViTForImageClassification.from_pretrained(
    MODEL_NAME, num_labels=len(IMG_CLASSES), id2label=id2label, label2id=label2id
)


/Users/mmenendezg/Developer/Projects/pneumonia-xray/.venv/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


ImportError: 
TFViTForImageClassification requires the TensorFlow library but it was not found in your environment.
However, we were able to find a PyTorch installation. PyTorch classes do not begin
with "TF", but are otherwise identically named to our TF classes.
If you want to use PyTorch, please use those classes instead!

If you really do want to use TensorFlow, please follow the instructions on the
installation page https://www.tensorflow.org/install that match your environment.


In [ ]:
label2id